In [757]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests

from pandas.io.json import json_normalize

from bs4 import BeautifulSoup

import re

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

import matplotlib.cm as cm
import matplotlib.colors as colors

In [557]:
CLIENT_ID = 'Y4KGOSYCDO52AJI54F5IILYD0QEYP5FL5VPCDNE5PXNBXZHH' # your Foursquare ID
CLIENT_SECRET = '40I1SND3VBE1SN0UWLZS4JN1ZYN4KRYJODFSSM5NW0M0RZV0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [8]:
url = 'https://api.foursquare.com/v2/venues/categories?client_id={}&client_secret={}&v={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION)

categories = requests.get(url).json()

In [9]:
def extract_values(obj, key):
    """Pull all values of specified key from nested JSON."""
    arr = []

    def extract(obj, arr, key):
        """Recursively search for values of key in JSON tree."""
        if isinstance(obj, dict):
            for k, v in obj.items():
                if isinstance(v, (dict, list)):
                    extract(v, arr, key)
                elif k == key:
                    arr.append(v)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr

    results = extract(obj, arr, key)
    return results

child_arr = extract_values(categories, 'name')

In [10]:
parent_arr = []
for parent in categories['response']['categories']:
    parent_arr.append(parent['name'])
parent_arr

['Arts & Entertainment',
 'College & University',
 'Event',
 'Food',
 'Nightlife Spot',
 'Outdoors & Recreation',
 'Professional & Other Places',
 'Residence',
 'Shop & Service',
 'Travel & Transport']

In [11]:
df_categories = pd.DataFrame(columns=['Venue Category', 'Venue'])
current_parent = ''
for cat in child_arr:
    if cat in parent_arr:
        current_parent = cat
    df_categories = df_categories.append({'Venue Category': current_parent, 'Venue': cat}, ignore_index=True) 
df_categories

,Venue Category,Venue
0,Arts & Entertainment,Arts & Entertainment
1,Arts & Entertainment,Amphitheater
2,Arts & Entertainment,Aquarium
3,Arts & Entertainment,Arcade
4,Arts & Entertainment,Art Gallery
5,Arts & Entertainment,Bowling Alley
6,Arts & Entertainment,Casino
7,Arts & Entertainment,Circus
8,Arts & Entertainment,Comedy Club
9,Arts & Entertainment,Concert Hall


In [12]:
df_categories['Venue Category'].value_counts()

Food                           348
Shop & Service                 173
Outdoors & Recreation          108
Professional & Other Places    106
Arts & Entertainment            65
Travel & Transport              54
College & University            39
Nightlife Spot                  25
Event                           13
Residence                        6
Name: Venue Category, dtype: int64

In [191]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population').text
soup = BeautifulSoup(source, 'lxml')
table = soup.find('table', class_='wikitable sortable')

<table class="wikitable sortable" style="text-align:center">
 <tbody>
  <tr>
   <th>
    2018
    <br/>
    rank
   </th>
   <th>
    City
   </th>
   <th>
    State
    <sup class="reference" id="cite_ref-5">
     <a href="#cite_note-5">
      [c]
     </a>
    </sup>
   </th>
   <th>
    2018
    <br/>
    estimate
   </th>
   <th>
    2010
    <br/>
    Census
   </th>
   <th>
    Change
   </th>
   <th colspan="2">
    2016 land area
   </th>
   <th colspan="2">
    2016 population density
   </th>
   <th>
    Location
   </th>
  </tr>
  <tr>
   <td>
    1
   </td>
   <td style="text-align:left;background-color:#cfecec">
    <i>
     <a href="/wiki/New_York_City" title="New York City">
      New York
     </a>
    </i>
    <sup class="reference" id="cite_ref-6">
     <a href="#cite_note-6">
      [d]
     </a>
    </sup>
   </td>
   <td align="left">
    <span class="flagicon">
     <img alt="" class="thumbborder" data-file-height="450" data-file-width="900" decoding="async" height

In [150]:
column_name = ''
for header in table.find_all('tr')[0].find_all('th'):
    column_name = column_name + header.text
column_name = column_name.split('\n')[:-1]

In [151]:
column_name[0] = 'Population Rank'
column_name[2] = 'State'
column_name[3] = 'Population (Latest Estimate)'
column_name[4] = 'Population (Latest Census)'
column_name[5] = 'Percent Change'
column_name[6] = 'Land Area (sq km)'
column_name[7] = 'Population density (sq km)'
column_name[8] = 'Latitude Longitude'

In [152]:
rows = []
for row in table.find_all('tr')[1:]:
    single_row = ''
    for i, column in enumerate(row.find_all('td')):
        text_add = ''
        if i == 1 or i ==2:
            try:
                text_add = column.find('a').text + '\n'
            except:
                text_add = column.text.lstrip()
        elif i == 7 or i == 9:
            text_add = re.findall('\d*[,.]*\d*[.]*\d*', column.text)[0] + '\n'
        elif i == 6 or i == 8:
            text_add = ''
        elif i == 10:
            str_ll = column.text
            str_ll = str_ll[str_ll.rfind("/")+2:]
            text_add = str_ll[:str_ll.find("(")-2]
        else:
            text_add = column.text
        single_row = single_row + text_add
    rows.append(single_row.split('\n'))

In [215]:
df_city = pd.DataFrame(rows, columns=column_name)

df_city['Population Rank'] = pd.to_numeric(df_city['Population Rank'])
df_city['Population (Latest Estimate)'] = df_city['Population (Latest Estimate)'].str.replace(',', '').astype(int)
df_city['Population (Latest Census)'] = df_city['Population (Latest Census)'].str.replace(',', '').astype(int)
df_city['Percent Change'] = round(100*(df_city['Population (Latest Estimate)'] / df_city['Population (Latest Census)'] - 1),2)
df_city['Land Area (sq mi)'] = df_city['Land Area (sq mi)'].str.replace(',', '').astype(float)
df_city['Population density (sq mi)'] = df_city['Population density (sq mi)'].str.replace(',', '').astype(float)
lat_long = df_city['Latitude Longitude'].str.split(";", expand=True)
df_city['Latitude'] = lat_long[0].astype(float)
df_city['Longitude'] = lat_long[1].astype(float)
df_city.drop(columns=['Latitude Longitude'], inplace=True)
df_city

,Population Rank,City,State,Population (Latest Estimate),Population (Latest Census),Percent Change,Land Area (sq mi),Population density (sq mi),Latitude,Longitude
0,1,New York,New York,8398748,8175133,2.74,301.5,28317.0,40.663500,-73.938700
1,2,Los Angeles,California,3990456,3792621,5.22,468.7,8484.0,34.019400,-118.410800
2,3,Chicago,Illinois,2705994,2695598,0.39,227.3,11900.0,41.837600,-87.681800
3,4,Houston,Texas,2325502,2100263,10.72,637.5,3613.0,29.786600,-95.390900
4,5,Phoenix,Arizona,1660272,1445632,14.85,517.6,3120.0,33.572200,-112.090100
5,6,Philadelphia,Pennsylvania,1584138,1526006,3.81,134.2,11683.0,40.009400,-75.133300
6,7,San Antonio,Texas,1532233,1327407,15.43,461.0,3238.0,29.472400,-98.525100
7,8,San Diego,California,1425976,1307402,9.07,325.2,4325.0,32.815300,-117.135000
8,9,Dallas,Texas,1345047,1197816,12.29,340.9,3866.0,32.793300,-96.766500
9,10,San Jose,California,1030119,945942,8.90,177.5,5777.0,37.296700,-121.818900


In [343]:
def city_data(cities, states):
    
    cities_list = []
    
    for city, state in zip(cities, states):
        
        city_url = 'https://www.census.gov/quickfacts/fact/table/{}city{}/PST045218'.format(city.replace(' ','').lower(),state.replace(' ','').lower())
        city_source = requests.get(city_url).text
        city_soup = BeautifulSoup(city_source, 'lxml')
        city_table = city_soup.find('div', class_='qf-facttable')
        
        city_row = [city, state]
        headers = ['City', 'State']
        counter = 0
        
        for i, tag in enumerate(city_table.find_all('td')):
            if counter == 0:
                if i % 2 == 0:
                    headers.append(re.sub('[\s]', '', tag.text))
            if i % 2 != 0:
                city_row.append(re.sub('[\s,%$]', '', tag.text.replace('\ue840\ue83f','')))
        
        counter = counter + 1
        cities_list.append(city_row)
        print(city, state)
    return [cities_list, headers]

In [345]:
# city_data_output = city_data(df_city['City'], df_city['State'])

New York New York
Los Angeles California
Chicago Illinois
Houston Texas
Phoenix Arizona
Philadelphia Pennsylvania
San Antonio Texas
San Diego California
Dallas Texas
San Jose California
Austin Texas
Jacksonville Florida
Fort Worth Texas
Columbus Ohio
San Francisco California
Charlotte North Carolina
Indianapolis Indiana
Seattle Washington
Denver Colorado
Washington District of Columbia
Boston Massachusetts
El Paso Texas
Detroit Michigan
Nashville Tennessee
Portland Oregon
Memphis Tennessee
Oklahoma City Oklahoma
Las Vegas Nevada
Louisville Kentucky
Baltimore Maryland
Milwaukee Wisconsin
Albuquerque New Mexico
Tucson Arizona
Fresno California
Mesa Arizona
Sacramento California
Atlanta Georgia
Kansas City Missouri
Colorado Springs Colorado
Miami Florida
Raleigh North Carolina
Omaha Nebraska
Long Beach California
Virginia Beach Virginia
Oakland California
Minneapolis Minnesota
Tulsa Oklahoma
Arlington Texas
Tampa Florida
New Orleans Louisiana
Wichita Kansas
Cleveland Ohio
Bakersfield Cali

In [417]:
# df_city_data = pd.DataFrame(city_data_output[0], columns=city_data_output[1])
# df_city_data.iloc[:,2:] = df_city_data.iloc[:,2:].apply(pd.to_numeric, errors='coerce')

# #Remove duplicate column for column 'Populationestimates,July1,2018,(V2018)' 
# df_city_data = df_city_data.loc[:,~df_city_data.columns.duplicated()]

In [440]:
#df_city_data

In [419]:
# missing_temp = df_city_data.loc[df_city_data['Populationestimates,July1,2018,(V2018)'] == 327167434][['City', 'State']]
# missing_temp.reset_index(inplace=True)
# missing_temp

,index,City,State
0,16,Indianapolis,Indiana
1,23,Nashville,Tennessee
2,28,Louisville,Kentucky
3,55,Honolulu,Hawaii
4,59,Lexington,Kentucky
5,62,Saint Paul,Minnesota
6,63,St. Louis,Missouri
7,67,Anchorage,Alaska
8,78,St. Petersburg,Florida
9,87,Gilbert,Arizona


In [372]:
# missing_city_types = ['indianapoliscitybalanceindiana',
#                       'nashvilledavidsonbalancetennessee',
#                       'louisvillejeffersoncountybalancekentucky',
#                       'honolulucountyhawaii',
#                       'lexingtonfayettekentucky',
#                       'stpaulcityminnesota',
#                       'stlouiscitymissouri',
#                       'anchoragemunicipalityalaska',
#                       'stpetersburgcityflorida',
#                       'gilberttownarizona',
#                       'winstonsalemcitynorthcarolina',
#                       'boisecitycityidaho',
#                       'augustarichmondcountybalancegeorgia',
#                       'portstluciecityflorida',
#                       'carytownnorthcarolina',
#                       'maconbibbcountygeorgia',
#                       'athensclarkecountybalancegeorgia',
#                       'sanbuenaventuraventuracitycalifornia',
#                       'davietownflorida',
#                       'lakewoodtownshipoceancountynewjersey',
#                       'clintonchartertownshipmacombcountymichigan',
#                       'edisontownshipmiddlesexcountynewjersey',
#                       'woodbridgetownshipmiddlesexcountynewjersey'
#                      ]

23

In [395]:
def city_data_missing(missing):
    
    cities_list = []
    
    for city in missing:
        
        city_url = 'https://www.census.gov/quickfacts/fact/table/{}/PST045218'.format(city)
        city_source = requests.get(city_url).text
        city_soup = BeautifulSoup(city_source, 'lxml')
        city_table = city_soup.find('div', class_='qf-facttable')
        
        city_row = []
        headers = []
        counter = 0
        
        for i, tag in enumerate(city_table.find_all('td')):
            if counter == 0:
                if i % 2 == 0:
                    headers.append(re.sub('[\s]', '', tag.text))
            if i % 2 != 0:
                city_row.append(re.sub('[\s,%$]', '', tag.text.replace('\ue840\ue83f','')))
        
        counter = counter + 1
        cities_list.append(city_row)
        print(city)
    return [cities_list, headers]

In [396]:
# city_data_output_missing = city_data_missing(missing_city_types)

indianapoliscitybalanceindiana
nashvilledavidsonbalancetennessee
louisvillejeffersoncountybalancekentucky
honolulucountyhawaii
lexingtonfayettekentucky
stpaulcityminnesota
stlouiscitymissouri
anchoragemunicipalityalaska
stpetersburgcityflorida
gilberttownarizona
winstonsalemcitynorthcarolina
boisecitycityidaho
augustarichmondcountybalancegeorgia
portstluciecityflorida
carytownnorthcarolina
maconbibbcountygeorgia
athensclarkecountybalancegeorgia
sanbuenaventuraventuracitycalifornia
davietownflorida
lakewoodtownshipoceancountynewjersey
clintonchartertownshipmacombcountymichigan
edisontownshipmiddlesexcountynewjersey
woodbridgetownshipmiddlesexcountynewjersey


In [397]:
# df_city_data_missing = pd.DataFrame(city_data_output_missing[0], columns=city_data_output_missing[1])
# df_city_data_missing.iloc[:,2:] = df_city_data_missing.iloc[:,2:].apply(pd.to_numeric, errors='coerce')
# # Remove duplicate columns
# df_city_data_missing = df_city_data_missing.loc[:,~df_city_data_missing.columns.duplicated()]
# df_city_data_missing

,"Populationestimates,July1,2018,(V2018)","Populationestimatesbase,April1,2010,(V2018)","Population,percentchange-April1,2010(estimatesbase)toJuly1,2018,(V2018)","Population,Census,April1,2010","Personsunder5years,percent","Personsunder18years,percent","Persons65yearsandover,percent","Femalepersons,percent","Whitealone,percent","BlackorAfricanAmericanalone,percent(a)","AmericanIndianandAlaskaNativealone,percent(a)","Asianalone,percent(a)","NativeHawaiianandOtherPacificIslanderalone,percent(a)","TwoorMoreRaces,percent","HispanicorLatino,percent(b)","Whitealone,notHispanicorLatino,percent","Veterans,2013-2017","Foreignbornpersons,percent,2013-2017","Housingunits,July1,2018,(V2018)","Owner-occupiedhousingunitrate,2013-2017","Medianvalueofowner-occupiedhousingunits,2013-2017","Medianselectedmonthlyownercosts-withamortgage,2013-2017","Medianselectedmonthlyownercosts-withoutamortgage,2013-2017","Mediangrossrent,2013-2017","Buildingpermits,2018","Households,2013-2017","Personsperhousehold,2013-2017","Livinginsamehouse1yearago,percentofpersonsage1year+,2013-2017","LanguageotherthanEnglishspokenathome,percentofpersonsage5years+,2013-2017","Householdswithacomputer,percent,2013-2017","HouseholdswithabroadbandInternetsubscription,percent,2013-2017","Highschoolgraduateorhigher,percentofpersonsage25years+,2013-2017","Bachelor'sdegreeorhigher,percentofpersonsage25years+,2013-2017","Withadisability,underage65years,percent,2013-2017","Personswithouthealthinsurance,underage65years,percent","Incivilianlaborforce,total,percentofpopulationage16years+,2013-2017","Incivilianlaborforce,female,percentofpopulationage16years+,2013-2017","Totalaccommodationandfoodservicessales,2012($1,000)(c)","Totalhealthcareandsocialassistancereceipts/revenue,2012($1,000)(c)","Totalmanufacturersshipments,2012($1,000)(c)","Totalmerchantwholesalersales,2012($1,000)(c)","Totalretailsales,2012($1,000)(c)","Totalretailsalespercapita,2012(c)","Meantraveltimetowork(minutes),workersage16years+,2013-2017","Medianhouseholdincome(in2017dollars),2013-2017","Percapitaincomeinpast12months(in2017dollars),2013-2017","Personsinpoverty,percent","Totalemployerestablishments,2016","Totalemployment,2016","Totalannualpayroll,2016($1,000)","Totalemployment,percentchange,2015-2016","Totalnonemployerestablishments,2017","Allfirms,2012","Men-ownedfirms,2012","Women-ownedfirms,2012","Minority-ownedfirms,2012","Nonminority-ownedfirms,2012","Veteran-ownedfirms,2012","Nonveteran-ownedfirms,2012","Populationpersquaremile,2010","Landareainsquaremiles,2010",FIPSCode
0,867125,820436,5.7,820445,7.4,24.7,11.5,51.7,61.8,28.1,0.3,3,0,3,10.1,55.9,45024,9.3,NaN,53.4,123500,1124,409,840,NaN,333316,2.51,84.2,13.3,84,74.3,85.5,29.7,10.5,14.1,67.1,63.4,2.52952e+06,1.00484e+07,NaN,NaN,1.34166e+07,16071,23.1,44709,26232,20.1,NaN,NaN,NaN,NaN,NaN,69366,34325,27668,18991,47298,6467,59063,2270,361.43,1.836e+06
1,669053,603427,10.9,601222,6.9,21.4,11.2,51.8,63.1,27.8,0.3,3.6,0.1,2.5,10.4,55.6,30865,12.8,NaN,54,191400,1346,472,970,NaN,263527,2.4,81.2,17.3,89,79.1,87.8,38.5,8.7,15.3,70.5,66,2.45512e+06,8.46692e+06,6.20115e+06,1.64461e+07,9.52776e+06,15257,24.6,52858,31109,17.2,NaN,NaN,NaN,NaN,NaN,68228,38160,24115,16382,49247,5930,59207,1265.4,475.13,4.75201e+06
2,620118,595386,4.2,597337,6.6,22.8,14.1,51.7,70.5,23.2,0.1,2.5,0.1,2.9,5.2,66.5,38617,7.1,NaN,59.7,146900,1214,440,779,NaN,246271,2.44,84.4,9,84.5,76.1,88.5,28.7,11.1,8.6,65.3,61.2,1.55954e+06,5.48852e+06,2.66713e+07,1.02313e+07,7.92233e+06,13092,22.4,49439,28975,16.7,NaN,NaN,NaN,NaN,NaN,52983,28752,18027,10264,40468,4913,44606,1836.6,325.25,2.14801e+06
3,980080,953206,2.8,953207,6.3,21.2,17.7,49.7,21.7,2.8,0.3,43,9.6,22.6,10,18,78413,19.4,352527,55.6,626400,2466,574,1653,2410,311451,3.06,85.4,28,90,83.8,91.4,34,6.3,4.1,61.2,58.6,5.27322e+06,6.30264e+06,NaN,8.0528e+06,1.30364e+07,13352,29.1,80078,33776,8.3,21404,359766,1.63689e+07,1.2,68673,76176,39248,28490,54725,17672,6943,65269,1586.7,600.74,15003
4,323780,295867,9.4,295803,6.2,21.1,12.2,51,75.6,14.5,0.3,3.6,

In [423]:
# df_city_data.loc[df_city_data['Populationestimates,July1,2018,(V2018)'] == 327167434] = pd.concat([missing_temp, df_city_data_missing], axis=1).set_index('index')

In [431]:
# df_city_data.to_csv('C:/Users/btmcc/Desktop/Desktop/IBM Data Science/9. Applied Data Science Capstone/city_data.csv', index=False)

In [576]:
df_city_final = pd.read_csv('C:/Users/btmcc/Desktop/Desktop/IBM Data Science/9. Applied Data Science Capstone/city_data.csv')

In [577]:
df_city_final.isna().sum()

City                                                                           0
State                                                                          0
Populationestimates,July1,2018,(V2018)                                         0
Populationestimatesbase,April1,2010,(V2018)                                    0
Population,percentchange-April1,2010(estimatesbase)toJuly1,2018,(V2018)        1
Population,Census,April1,2010                                                  1
Personsunder5years,percent                                                     0
Personsunder18years,percent                                                    0
Persons65yearsandover,percent                                                  0
Femalepersons,percent                                                          0
Whitealone,percent                                                             0
BlackorAfricanAmericanalone,percent(a)                                         0
AmericanIndianandAlaskaNativ

In [578]:
#Data cleaning
df_city_final.loc[180,'Population,percentchange-April1,2010(estimatesbase)toJuly1,2018,(V2018)'] = 100*(df_city_final.loc[180,'Populationestimates,July1,2018,(V2018)']/df_city_final.loc[180,'Population,Census,April1,2010']-1)
df_city_final.loc[218,['Allfirms,2012']] = 25

In [583]:
#Three cities dropped due to NaN values
df_city_final.drop(df_city_final[df_city_final['Totalretailsales,2012($1,000)(c)'].isna() | df_city_final['Totalretailsalespercapita,2012(c)'].isna() | df_city_final['Populationpersquaremile,2010'].isna() | df_city_final['Landareainsquaremiles,2010'].isna()].index,inplace=True)
df_city_final.reset_index(drop=True, inplace=True)

In [584]:
df_city_final.isna().sum()


City                                                                           0
State                                                                          0
Populationestimates,July1,2018,(V2018)                                         0
Populationestimatesbase,April1,2010,(V2018)                                    0
Population,percentchange-April1,2010(estimatesbase)toJuly1,2018,(V2018)        0
Population,Census,April1,2010                                                  0
Personsunder5years,percent                                                     0
Personsunder18years,percent                                                    0
Persons65yearsandover,percent                                                  0
Femalepersons,percent                                                          0
Whitealone,percent                                                             0
BlackorAfricanAmericanalone,percent(a)                                         0
AmericanIndianandAlaskaNativ

In [585]:
df_city_final = df_city_final.loc[:,~df_city_final.isna().any()]
df_city_final.isna().sum()

City                                                                         0
State                                                                        0
Populationestimates,July1,2018,(V2018)                                       0
Populationestimatesbase,April1,2010,(V2018)                                  0
Population,percentchange-April1,2010(estimatesbase)toJuly1,2018,(V2018)      0
Population,Census,April1,2010                                                0
Personsunder5years,percent                                                   0
Personsunder18years,percent                                                  0
Persons65yearsandover,percent                                                0
Femalepersons,percent                                                        0
Whitealone,percent                                                           0
BlackorAfricanAmericanalone,percent(a)                                       0
AmericanIndianandAlaskaNativealone,percent(a)       

In [561]:
def reviseCityNames(cities, states):
    
    revise=[]
    errors=[]
    for city, state in zip(cities, states):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}, {}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            city, 
            state, 
            100000)
        
        try:
            results = requests.get(url).json()
            if results['meta']['code'] == 400:
                revise.append([city, state])
                print(city, state)
        
        except:
            errors.append([city, state])
    
    return [revise, errors]

In [563]:
revise_cities = reviseCityNames(df_city_final['City'],df_city_final['State'])
#revise_cities = reviseCityNames(['New York', 'Winston–Salem'],['New York', 'North Carolina'])

Winston–Salem North Carolina
Jurupa Valley California


In [565]:
revise_cities[1]

[['Centennial', 'Colorado']]

In [596]:
# update city name of Winston-Salem
df_city_final.iloc[87,0] = 'Winston Salem'

In [604]:
errors = []
def getRecommendedVenues(cities, states):
    
    venues_list=[]
    
    for city, state in zip(cities, states):
        print(city, state)
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}, {}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            city, 
            state, 
            100000)
        
        try:
            # make the GET request
            results = requests.get(url).json()["response"]['groups'][0]['items']

            # return only relevant information for each nearby venue
            venues_list.append([( 
                city, 
                state, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])
        
        except:
            errors.append([city, state])
    rec_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    rec_venues.columns = ['City', 
                  'State',  
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(rec_venues)

In [605]:
city_venues = getRecommendedVenues(df_city_final['City'],df_city_final['State'])

New York New York
Los Angeles California
Chicago Illinois
Houston Texas
Phoenix Arizona
Philadelphia Pennsylvania
San Antonio Texas
San Diego California
Dallas Texas
San Jose California
Austin Texas
Jacksonville Florida
Fort Worth Texas
Columbus Ohio
San Francisco California
Charlotte North Carolina
Indianapolis Indiana
Seattle Washington
Denver Colorado
Washington District of Columbia
Boston Massachusetts
El Paso Texas
Detroit Michigan
Nashville Tennessee
Portland Oregon
Memphis Tennessee
Oklahoma City Oklahoma
Las Vegas Nevada
Louisville Kentucky
Baltimore Maryland
Milwaukee Wisconsin
Albuquerque New Mexico
Tucson Arizona
Fresno California
Mesa Arizona
Sacramento California
Atlanta Georgia
Kansas City Missouri
Colorado Springs Colorado
Miami Florida
Raleigh North Carolina
Omaha Nebraska
Long Beach California
Virginia Beach Virginia
Oakland California
Minneapolis Minnesota
Tulsa Oklahoma
Arlington Texas
Tampa Florida
New Orleans Louisiana
Wichita Kansas
Cleveland Ohio
Bakersfield Cali

In [629]:
# Check to see which cities are missing from city_venues
city_missing_venue = df_city_final.loc[~df_city_final['City'].isin(city_venues['City'])]
city_missing_venue

,City,State,"Populationestimates,July1,2018,(V2018)","Populationestimatesbase,April1,2010,(V2018)","Population,percentchange-April1,2010(estimatesbase)toJuly1,2018,(V2018)","Population,Census,April1,2010","Personsunder5years,percent","Personsunder18years,percent","Persons65yearsandover,percent","Femalepersons,percent","Whitealone,percent","BlackorAfricanAmericanalone,percent(a)","AmericanIndianandAlaskaNativealone,percent(a)","Asianalone,percent(a)","NativeHawaiianandOtherPacificIslanderalone,percent(a)","TwoorMoreRaces,percent","HispanicorLatino,percent(b)","Whitealone,notHispanicorLatino,percent","Veterans,2013-2017","Foreignbornpersons,percent,2013-2017","Owner-occupiedhousingunitrate,2013-2017","Medianvalueofowner-occupiedhousingunits,2013-2017","Medianselectedmonthlyownercosts-withamortgage,2013-2017","Medianselectedmonthlyownercosts-withoutamortgage,2013-2017","Mediangrossrent,2013-2017","Households,2013-2017","Personsperhousehold,2013-2017","Livinginsamehouse1yearago,percentofpersonsage1year+,2013-2017","LanguageotherthanEnglishspokenathome,percentofpersonsage5years+,2013-2017","Householdswithacomputer,percent,2013-2017","HouseholdswithabroadbandInternetsubscription,percent,2013-2017","Highschoolgraduateorhigher,percentofpersonsage25years+,2013-2017","Bachelor'sdegreeorhigher,percentofpersonsage25years+,2013-2017","Withadisability,underage65years,percent,2013-2017","Personswithouthealthinsurance,underage65years,percent","Incivilianlaborforce,total,percentofpopulationage16years+,2013-2017","Incivilianlaborforce,female,percentofpopulationage16years+,2013-2017","Totalretailsales,2012($1,000)(c)","Totalretailsalespercapita,2012(c)","Meantraveltimetowork(minutes),workersage16years+,2013-2017","Medianhouseholdincome(in2017dollars),2013-2017","Percapitaincomeinpast12months(in2017dollars),2013-2017","Personsinpoverty,percent","Allfirms,2012","Populationpersquaremile,2010","Landareainsquaremiles,2010",FIPSCode
49,New Orleans,Louisiana,391006.0,343828.0,13.7,343829.0,6.0,20.3,12.9,52.3,34.1,59.8,0.2,3.0,0.0,1.8,5.5,30.6,16793.0,5.9,47.1,205000.0,1653.0,481.0,954.0,154895.0,2.42,84.8,8.7,79.1,66.1,85.9,36.5,10.3,14.1,61.4,59.5,3245075.0,8788.0,23.8,38721.0,29275.0,25.4,41506.0,2029.4,169.42,2255000.0
55,Honolulu,Hawaii,980080.0,953206.0,2.8,953207.0,6.3,21.2,17.7,49.7,21.7,2.8,0.3,43.0,9.6,22.6,10.0,18.0,78413.0,19.4,55.6,626400.0,2466.0,574.0,1653.0,311451.0,3.06,85.4,28.0,90.0,83.8,91.4,34.0,6.3,4.1,61.2,58.6,13036448.0,13352.0,29.1,80078.0,33776.0,8.3,76176.0,1586.7,600.74,15003.0


In [630]:
city_venues_missing = getRecommendedVenues(city_missing_venue['City'],city_missing_venue['State'])

New Orleans Louisiana
Honolulu Hawaii


In [634]:
city_venues_final = pd.concat([city_venues, city_venues_missing], axis=0, ignore_index=True)

In [642]:
# Confirm all cities are captured in the final venue datafram
city_venues_final.groupby(['City', 'State']).count().shape[0]

311

In [643]:
city_venues_final.head()

,City,State,Venue,Venue Latitude,Venue Longitude,Venue Category
0,New York,New York,Central Park,40.784083,-73.964853,Park
1,New York,New York,Brooklyn Bridge Park,40.702282,-73.996456,Park
2,New York,New York,Los Tacos No. 1,40.757134,-73.987536,Taco Place
3,New York,New York,The Metropolitan Museum of Art (Metropolitan M...,40.779729,-73.963416,Art Museum
4,New York,New York,Hudson River Greenway Running Path,40.732552,-74.010580,Trail


In [663]:
# Classify venues by venue category
city_venue_cat = city_venues_final.merge(df_categories, left_on='Venue Category', right_on='Venue')

In [664]:
city_venue_cat.head()

,City,State,Venue_x,Venue Latitude,Venue Longitude,Venue Category_x,Venue Category_y,Venue_y
0,New York,New York,Central Park,40.784083,-73.964853,Park,Outdoors & Recreation,Park
1,New York,New York,Brooklyn Bridge Park,40.702282,-73.996456,Park,Outdoors & Recreation,Park
2,New York,New York,Hudson River Park,40.733217,-74.010494,Park,Outdoors & Recreation,Park
3,New York,New York,Bryant Park,40.753621,-73.983265,Park,Outdoors & Recreation,Park
4,New York,New York,High Line,40.746825,-74.005507,Park,Outdoors & Recreation,Park


In [666]:
city_venue_cat.rename(columns={'Venue Category_y':'Venue Category'}, inplace=True)
city_venue_cat = city_venue_cat[['City', 'State', 'Venue Category']]
city_venue_cat.head()

,City,State,Venue Category
0,New York,New York,Outdoors & Recreation
1,New York,New York,Outdoors & Recreation
2,New York,New York,Outdoors & Recreation
3,New York,New York,Outdoors & Recreation
4,New York,New York,Outdoors & Recreation


In [671]:
# one hot encoding
venue_onehot = pd.get_dummies(city_venue_cat[['Venue Category']], prefix='', prefix_sep='')
df_city_venue_cat = pd.concat([city_venue_cat[['City', 'State']], venue_onehot], axis=1)
df_city_venue_cat.head()

,City,State,Arts & Entertainment,College & University,Event,Food,Nightlife Spot,Outdoors & Recreation,Professional & Other Places,Residence,Shop & Service,Travel & Transport
0,New York,New York,0,0,0,0,0,1,0,0,0,0
1,New York,New York,0,0,0,0,0,1,0,0,0,0
2,New York,New York,0,0,0,0,0,1,0,0,0,0
3,New York,New York,0,0,0,0,0,1,0,0,0,0
4,New York,New York,0,0,0,0,0,1,0,0,0,0


In [681]:
df_city_venue_grouped = df_city_venue_cat.groupby(['City', 'State']).mean().reset_index()
df_city_venue_grouped.head()

,City,State,Arts & Entertainment,College & University,Event,Food,Nightlife Spot,Outdoors & Recreation,Professional & Other Places,Residence,Shop & Service,Travel & Transport
0,Abilene,Texas,0.053191,0.00,0.0,0.691489,0.021277,0.00,0.00,0.0,0.202128,0.031915
1,Akron,Ohio,0.110000,0.00,0.0,0.510000,0.100000,0.12,0.01,0.0,0.150000,0.000000
2,Albuquerque,New Mexico,0.090000,0.01,0.0,0.520000,0.080000,0.08,0.01,0.0,0.180000,0.030000
3,Alexandria,Virginia,0.030000,0.00,0.0,0.470000,0.030000,0.21,0.01,0.0,0.230000,0.020000
4,Allen,Texas,0.040000,0.00,0.0,0.590000,0.030000,0.10,0.00,0.0,0.230000,0.010000


In [760]:
df_final = df_city_venue_grouped.merge(df_city_final[['City',
 'State',
 'Populationestimates,July1,2018,(V2018)',
 'Population,percentchange-April1,2010(estimatesbase)toJuly1,2018,(V2018)',
 'Mediangrossrent,2013-2017',
 "Bachelor'sdegreeorhigher,percentofpersonsage25years+,2013-2017",
 'Incivilianlaborforce,total,percentofpopulationage16years+,2013-2017',
 'Percapitaincomeinpast12months(in2017dollars),2013-2017',
 'Landareainsquaremiles,2010'
]])
df_cluster = df_final.drop(['City','State'], axis=1)
df_cluster.head()

,Arts & Entertainment,College & University,Event,Food,Nightlife Spot,Outdoors & Recreation,Professional & Other Places,Residence,Shop & Service,Travel & Transport,"Populationestimates,July1,2018,(V2018)","Population,percentchange-April1,2010(estimatesbase)toJuly1,2018,(V2018)","Mediangrossrent,2013-2017","Bachelor'sdegreeorhigher,percentofpersonsage25years+,2013-2017","Incivilianlaborforce,total,percentofpopulationage16years+,2013-2017","Percapitaincomeinpast12months(in2017dollars),2013-2017","Landareainsquaremiles,2010"
0,0.053191,0.00,0.0,0.691489,0.021277,0.00,0.00,0.0,0.202128,0.031915,122999.0,4.7,846.0,22.1,57.5,22819.0,106.79
1,0.110000,0.00,0.0,0.510000,0.100000,0.12,0.01,0.0,0.150000,0.000000,198006.0,-0.6,713.0,20.8,62.3,22416.0,62.03
2,0.090000,0.01,0.0,0.520000,0.080000,0.08,0.01,0.0,0.180000,0.030000,560218.0,2.6,833.0,34.3,63.8,28229.0,187.73
3,0.030000,0.00,0.0,0.470000,0.030000,0.21,0.01,0.0,0.230000,0.020000,160530.0,14.7,1663.0,61.8,76.9,57019.0,15.03
4,0.040000,0.00,0.0,0.590000,0.030000,0.10,0.00,0.0,0.230000,0.010000,103383.0,22.7,1374.0,54.3,73.4,42098.0,26.29


In [752]:
X = StandardScaler().fit_transform(df_cluster)

kclusters = 5

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=123).fit(X)

,Arts & Entertainment,College & University,Event,Food,Nightlife Spot,Outdoors & Recreation,Professional & Other Places,Residence,Shop & Service,Travel & Transport,"Populationestimates,July1,2018,(V2018)","Population,percentchange-April1,2010(estimatesbase)toJuly1,2018,(V2018)","Mediangrossrent,2013-2017","Bachelor'sdegreeorhigher,percentofpersonsage25years+,2013-2017","Incivilianlaborforce,total,percentofpopulationage16years+,2013-2017","Percapitaincomeinpast12months(in2017dollars),2013-2017","Landareainsquaremiles,2010"
Cluster,,,,,,,,,,,,,,,,,
0,0.053247,0.000984,0.00,0.525307,0.048946,0.143607,0.010830,0.000328,0.202102,0.014650,2.472814e+05,13.595475,1470.016393,49.826230,69.081967,42738.918033,56.325410
1,0.077155,0.007426,0.00,0.547281,0.093439,0.092659,0.006327,0.000090,0.156489,0.019134,2.499803e+05,5.995495,897.666667,32.240541,64.267568,26906.279279,89.085405
2,0.156818,0.001364,0.00,0.401364,0.067273,0.167727,0.005909,0.000909,0.161364,0.037273,1.510254e+06,8.700000,1078.136364,31.681818,65.895455,29760.090909,415.149091
3,0.100000,0.005000,0.01,0.560000,0.050000,0.065000,0.005000,0.000000,0.190000,0.015000,1.658345e+05,-0.600000,861.500000,37.050000,58.850000,26789.500000,145.890000
4,0.034726,0.000531,0.00,0.592321,0.036056,0.076400,0.003623,0.000000,0.244323,0.012020,1.622543e+05,7.844348,1137.947826,24.200000,63.457391,25551.773913,56.539217


In [758]:
df_cluster['Cluster'] = kmeans.labels_
df_cluster.groupby('Cluster').mean()

,Arts & Entertainment,College & University,Event,Food,Nightlife Spot,Outdoors & Recreation,Professional & Other Places,Residence,Shop & Service,Travel & Transport,"Populationestimates,July1,2018,(V2018)","Population,percentchange-April1,2010(estimatesbase)toJuly1,2018,(V2018)","Mediangrossrent,2013-2017","Bachelor'sdegreeorhigher,percentofpersonsage25years+,2013-2017","Incivilianlaborforce,total,percentofpopulationage16years+,2013-2017","Percapitaincomeinpast12months(in2017dollars),2013-2017","Landareainsquaremiles,2010"
Cluster,,,,,,,,,,,,,,,,,
0,0.053247,0.000984,0.00,0.525307,0.048946,0.143607,0.010830,0.000328,0.202102,0.014650,2.472814e+05,13.595475,1470.016393,49.826230,69.081967,42738.918033,56.325410
1,0.077155,0.007426,0.00,0.547281,0.093439,0.092659,0.006327,0.000090,0.156489,0.019134,2.499803e+05,5.995495,897.666667,32.240541,64.267568,26906.279279,89.085405
2,0.156818,0.001364,0.00,0.401364,0.067273,0.167727,0.005909,0.000909,0.161364,0.037273,1.510254e+06,8.700000,1078.136364,31.681818,65.895455,29760.090909,415.149091
3,0.100000,0.005000,0.01,0.560000,0.050000,0.065000,0.005000,0.000000,0.190000,0.015000,1.658345e+05,-0.600000,861.500000,37.050000,58.850000,26789.500000,145.890000
4,0.034726,0.000531,0.00,0.592321,0.036056,0.076400,0.003623,0.000000,0.244323,0.012020,1.622543e+05,7.844348,1137.947826,24.200000,63.457391,25551.773913,56.539217


Cluster 0 - Medium Population; Outdoors/Recreation; Growing Cities; Higher Rent; Well-educated; High Income  
Cluster 1 - Medium Population; Nightlife; Lower Rent  
Cluster 2 - High Population; Emphasis on Arts/Entertainment + Outdoors/Recreation; Large land area  
Cluster 3 - Lower Population; Less Activities; Cities not growing; Lower Rent; Less population density  
Cluster 4 - Lower Population; Less Acitivies; Less educated; Smaller land area  

In [763]:
df_final['Cluster'] = kmeans.labels_
df_viz = df_final.merge(df_city[[
    'City',
    'State',
    'Latitude',
    'Longitude'
]])
df_viz.head()

,City,State,Arts & Entertainment,College & University,Event,Food,Nightlife Spot,Outdoors & Recreation,Professional & Other Places,Residence,Shop & Service,Travel & Transport,"Populationestimates,July1,2018,(V2018)","Population,percentchange-April1,2010(estimatesbase)toJuly1,2018,(V2018)","Mediangrossrent,2013-2017","Bachelor'sdegreeorhigher,percentofpersonsage25years+,2013-2017","Incivilianlaborforce,total,percentofpopulationage16years+,2013-2017","Percapitaincomeinpast12months(in2017dollars),2013-2017","Landareainsquaremiles,2010",Cluster,Latitude,Longitude
0,Abilene,Texas,0.053191,0.00,0.0,0.691489,0.021277,0.00,0.00,0.0,0.202128,0.031915,122999.0,4.7,846.0,22.1,57.5,22819.0,106.79,4,32.4545,-99.7381
1,Akron,Ohio,0.110000,0.00,0.0,0.510000,0.100000,0.12,0.01,0.0,0.150000,0.000000,198006.0,-0.6,713.0,20.8,62.3,22416.0,62.03,1,41.0805,-81.5214
2,Albuquerque,New Mexico,0.090000,0.01,0.0,0.520000,0.080000,0.08,0.01,0.0,0.180000,0.030000,560218.0,2.6,833.0,34.3,63.8,28229.0,187.73,1,35.1056,-106.6474
3,Alexandria,Virginia,0.030000,0.00,0.0,0.470000,0.030000,0.21,0.01,0.0,0.230000,0.020000,160530.0,14.7,1663.0,61.8,76.9,57019.0,15.03,0,38.8201,-77.0841
4,Allen,Texas,0.040000,0.00,0.0,0.590000,0.030000,0.10,0.00,0.0,0.230000,0.010000,103383.0,22.7,1374.0,54.3,73.4,42098.0,26.29,0,33.0997,-96.6631


In [783]:
# create map
map_clusters = folium.Map(location=[39.8283, -98.5795], zoom_start=4)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_viz['Latitude'], df_viz['Longitude'], df_viz['City'], df_viz['Cluster']):
    label = folium.Popup(poi + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters